In [25]:
import os 
kafka = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1"
spark = "org.apache.spark:spark-streaming-kafka-0-8_2.11:2.1.0"

os.environ["PYSPARK_SUBMIT_ARGS"] = ("--packages {},{} pyspark-shell".format(kafka, spark))

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [26]:
KAFKA_BROKER = "kafka-1:9092,kafka-2:9092,kafka-3:9092"

KAFKA_TOPIC = "tiki"

spark = SparkSession.builder.appName("tiki_analysis").master("spark://spark-master:7077").getOrCreate()


In [27]:
kafkaMessages = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", KAFKA_BROKER) \
  .option("subscribe", KAFKA_TOPIC) \
  .option("startingOffsets", "earliest") \
  .load()

In [28]:
message = kafkaMessages.selectExpr("CAST(value AS STRING)")

In [37]:
rawQuery = message \
        .writeStream \
        .queryName("qraw2")\
        .format("memory")\
        .start()

In [35]:
spark.sql("SELECT * FROM qraw2").show()

+--------------------+
|               value|
+--------------------+
|{"id": 32033717, ...|
|{"id": 76181490, ...|
|{"id": 4897521, "...|
|{"id": 54439144, ...|
|{"id": 73320913, ...|
|{"id": 38458616, ...|
|{"id": 53056161, ...|
|{"id": 99376042, ...|
|{"id": 53080935, ...|
|{"id": 70771651, ...|
|{"id": 81706382, ...|
|{"id": 67910637, ...|
|{"id": 416978, "s...|
|{"id": 62445418, ...|
|{"id": 58259141, ...|
|{"id": 73795655, ...|
|{"id": 9838905, "...|
|{"id": 22356225, ...|
|{"id": 74975883, ...|
|{"id": 12567795, ...|
+--------------------+
only showing top 20 rows



In [ ]:
fileStream = message.writeStream \
                      .trigger(processingTime='60 seconds')\
                      .queryName("Persist the processed data") \
                      .outputMode("append") \
                      .format("json") \
                      .option("path", "hdfs://namenode:9000/viethoang/inputHDFS/") \
                      .option("checkpointLocation", "hdfs://namenode:9000/viethoang/checkpoint") \
                      .start() \
                      .awaitTermination()